# Proyecto 03 - Procesamiento del Lenguaje Natural

## Dataset: The Multilingual Amazon Reviews Corpus

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Es un archivo .zip que contiene tres documentos. Más información sobre el dataset [aquí](https://registry.opendata.aws/amazon-reviews-ml/). Es importante que tengas en cuenta la [licencia](https://docs.opendata.aws/amazon-reviews-ml/license.txt) de este dataset.**

### Exploración de datos y Procesamiento del Lenguaje Natural

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Considera que hasta que no hayas aplicado las herramientas de Procesamiento del Lenguaje Natural vistas, será difícil completar este análisis. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿qué palabras están asociadas a calificaciones positivas y qué palabras a calificaciones negativas?

### Machine Learning

Implementa un modelo que, dada la crítica de un producto, asigne la cantidad de estrellas correspondiente. **Para pensar**: ¿es un problema de Clasificación o de Regresión?

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.
1. Intenta responder la pregunta: ¿Qué información está usando el modelo para predecir?

**Recomendación:** si no te resulta conveniente trabajar en español con NLTK, te recomendamos que explores la librería [spaCy](https://spacy.io/).

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta. Pensar en qué situación puede ser útil. ¿Esperas que el desempeño sea mejor o peor?
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

#####################################################################################################################

### 1. Análisis Exploratorio de Datos

In [2]:
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import json

In [5]:
dataset = pd.read_json('dataset_es_train.json', lines = True)
dataset.tail(20)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
199980,es_0538480,product_es_0824419,reviewer_es_0227416,5,"Juego entretenido,envío sin problema",Esplendor,es,toy
199981,es_0901817,product_es_0290920,reviewer_es_0699350,5,la diadema encaja perfectamente. Se ve casi id...,Encaja muy bien,es,electronics
199982,es_0955886,product_es_0676496,reviewer_es_0179032,5,Tal y como viene explicado en la descripción. ...,Bonito y práctico,es,home
199983,es_0891755,product_es_0321086,reviewer_es_0077312,5,La verdad que tenía una plaga de mosquitos en ...,Adiós mosquitos,es,lawn_and_garden
199984,es_0290234,product_es_0698567,reviewer_es_0888388,5,"Esta bien, precio súper barato",Contenta,es,baby_product
199985,es_0518177,product_es_0625125,reviewer_es_0714779,5,"Súper eficaz. Me llegó ayer, vi las instruccio...",Perfecto,es,home_improvement
199986,es_0957177,product_es_0062883,reviewer_es_0718985,5,Lo compré para mi sobrino de año y medio y le ...,Mejor de lo q esperaba,es,toy
199987,es_0034265,product_es_0007118,reviewer_es_0143807,5,Sartén antiadherente para todo tipo de comidas...,Sartén antiadherente,es,home
199988,es_0415999,product_es_0236136,reviewer_es_0823988,5,"Correcto en todos los sentido, fácil manejo pa...",Facilidad de uso,es,home_improvement
199989,es_0387212,product_es_0759866,reviewer_es_0081649,5,"llego de un día para otro, super fácil de mont...",calidad precio,es,home
